# Film Script Analyzer

## Summarization

Summarization is a method to extract meaningful information from a bigger source. In the case of scripts, it is to extract sentences that can uniquely identify the script, sentences that capture the essence of the script.

In order to do it, a statistical method is used. The frequencies of words are calculated, very high frequency words usually do not have a lot of information (like stopwords), very low frequency words may not convey the spirit of the script, they may very well be isolated instances.

In [1]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus   import stopwords
from collections   import defaultdict
from string        import punctuation
from heapq         import nlargest

def frequencies(word_sent,min_cut=0.1,max_cut=0.9):              #Set frequencies cut offs
    stopw = set(stopwords.words('english')+list(punctuation))    #Create set of stopwords and punctuation to remove
    freq  = defaultdict(int)                                     #Create dictionary for frequencies
    
    for s in word_sent:                                          #For each sentence in word sentences
        for word in s:                                           #For each word in sentence
            if word not in stopw:                                #If word is valid
                freq[word] += 1                                  #Count it
    m = float(max(freq.values()))                                #Get maximum value (for normalization)
    
    for w in freq.keys():                                        #For each word  
        freq[w] = freq[w]/m                                      #Normalize
        if freq[w] >= max_cut or freq[w] <= min_cut:             #Is word within the cutoff range?
            del freq[w]                                          #Delete if not
            
    return freq                                                  #Return dict

Then to summarize, set thresholds of minimum and maximum frequencies and then choose from among the remaining sentences the top ones.

In [9]:
def rank(ranking,n):                                             #Receives a list of sentences and frequencies
    return nlargest(n,ranking,key=ranking.get)                   #Returns highest rated

def summarize(text,n=5):                                         #Main function, receives text and #of sentences 
    sents      = sent_tokenize(text)                             #Splits text into sentences (token sentences) 
    assert n  <= len(sents)                                      #Number of total sentences to return at least n
    word_sent  = [word_tokenize(s.lower()) for s in sents]       #Tokenize words from sentences
    freq       = frequencies(word_sent)                          #Send bag of words to frequencies function
    ranking    = defaultdict(int)                                #Create rank dictionary
    
    for i,sent in enumerate(word_sent):                          #For each tokenized sentence
        for w in sent:                                           #For each word in sentence
            if w in freq:                                        #if word is in frequency dictionary
                ranking[i] += freq[w]                            #add its normalized frequency to the total
                
    sents_idx = rank(ranking,n)                                  #Send to rank, which will return n highest ranked sentences
    
    return [sents[j] for j in sents_idx]                         #Return all sentences

Open text and try it.

In [13]:
text = open('scripts/Eat Pray Love 2010.txt','r').readlines()[2]

summarize(text,10)

["I'm going to Italy and then l'm going to David's guru's ashram in lndia... ...and l'm going to end the year in Bali.",
 "It's long, it's tedious, I can't keep up... ...and l get these insane anxieties about everything in my life... ...and l've lost my place.",
 'And it was such a foreign concept to me, that l swear l almost began with: "l\'m a big fan of your work."',
 "-No, I don't even have my-- l-- You don't have your-- You don't-- You're so naked.",
 "And l-- You know, I don't-- I don't know.",
 "Do not tell me what lessons l have and haven't learned in the last year... ...and don't tell me how balanced and wise you are... ...and how I can't express myself.",
 "If it wasn't for you, I wouldn't have come back to Bali... ...and l wouldn't have come back to myself.",
 "l'm sorry l didn't call sooner.",
 "I don't know why we can't accept... ...we don't wanna live in unhappiness anymore.",
 "You know, it's been a rough day, and if no one takes it personally... ...l'm going to take my 

Next Section: [word2vec](https://github.com/luisecastro/film_script_analysis/blob/master/11_word2vec.ipynb)